In [51]:
import xarray as xr
import numpy as np

Conservative Coarseninf via Area Weighted Mean without paying any heed to the target grid here

In [56]:
from regridding_II import coarsen_file_conservatively

for RhiresD : coarsening to 0.11 degrees

In [60]:
coarsened_rhiresd = coarsen_file_conservatively(
    infile="RhiresD_1971_2022.nc",
    varname="RhiresD",
    block_size=11,
    outfile="RhiresD_011deg_coarsened.nc"
)


for TabsD : coarsening to 0.11 degrees

In [61]:
coarsened_tabsd = coarsen_file_conservatively(
    infile="TabsD_1971_2022.nc",
    varname="TabsD",
    block_size=11,
    outfile="TabsD_011deg_coarsened.nc"
)

In [62]:
print(coarsened_rhiresd.shape)

(18993, 24, 33)


In [63]:
print(coarsened_tabsd.shape)

(18993, 24, 33)
